In [1]:
import pandas as pd 
from datetime import datetime, date, time, timedelta

 
df = pd.read_csv("crime.csv") 
df.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,3152762.0,1667011.0,-104.957381,39.663490,3,314,wellshire,1,0
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0


In [2]:

df['DATE'], df['TIME'] = df['FIRST_OCCURRENCE_DATE'].str.split(' ', 1).str


df['DATE'] = pd.to_datetime(df['DATE'])
df['day_of_week'] = df['DATE'].dt.day_name()
df

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,...,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,DATE,TIME,day_of_week
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,...,-104.809881,39.773188,5,521,montbello,1,0,2016-06-15,11:31:00 PM,Wednesday
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,...,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0,2017-10-11,12:30:00 PM,Wednesday
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,...,-104.957381,39.663490,3,314,wellshire,1,0,2016-03-04,8:00:00 PM,Friday
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,...,-104.941440,39.702698,3,312,belcaro,1,0,2018-01-30,7:20:00 PM,Tuesday
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,...,-104.955370,39.717107,3,311,cherry-creek,1,0,2017-06-22,8:53:00 PM,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526095,2019688353,2019688353121200,1212,0,robbery-car-jacking,robbery,10/27/2019 11:45:00 PM,NaN,10/28/2019 12:08:00 AM,E COLFAX AVE / N XENIA ST,...,-104.885821,39.740155,2,223,east-colfax,1,0,2019-10-27,11:45:00 PM,Sunday
526096,2019688368,2019688368540100,5401,0,traffic-accident-hit-and-run,traffic-accident,10/28/2019 12:20:00 AM,NaN,10/28/2019 12:35:00 AM,1959 S BROADWAY ST,...,-104.987794,39.681001,3,313,overland,0,1,2019-10-28,12:20:00 AM,Monday
526097,2019688404,2019688404131302,1313,2,assault-dv,other-crimes-against-persons,10/28/2019 1:13:00 AM,NaN,10/28/2019 3:26:00 AM,7030 N TOWER RD,...,-104.770112,39.824869,5,522,dia,1,0,2019-10-28,1:13:00 AM,Monday
526098,2019688441,2019688441131302,1313,2,assault-dv,other-crimes-against-persons,10/28/2019 2:10:00 AM,NaN,10/28/2019 4:25:00 AM,3831 N SHOSHONE ST,...,-105.009777,39.769921,1,111,sunnyside,1,0,2019-10-28,2:10:00 AM,Monday


In [3]:
df_property = df[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID','GEO_LON','GEO_LAT']]
df_property = df_property.loc[(df_property.OFFENSE_CATEGORY_ID == 'arson')|(df_property.OFFENSE_CATEGORY_ID == 'theft-from-motor-vehicle')|(df_property.OFFENSE_CATEGORY_ID == 'larceny')|(df_property.OFFENSE_CATEGORY_ID == 'burglary')|(df_property.OFFENSE_CATEGORY_ID == 'auto-theft')].copy()
df_property_am =  df_property.loc[df_property.TIME.str[-2:] == "AM" ].copy()
df_property_am['TIME'] = [x[:-3] for x in df_property_am['TIME']]
df_property_am['TIME'] =  pd.to_datetime(df_property_am['TIME'], format='%H:%M:%S')


In [4]:
df_property = df_property[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID','GEO_LON','GEO_LAT']]
df_property = df_property.loc[(df_property.OFFENSE_CATEGORY_ID == 'arson')|(df_property.OFFENSE_CATEGORY_ID == 'theft-from-motor-vehicle')|(df_property.OFFENSE_CATEGORY_ID == 'larceny')|(df_property.OFFENSE_CATEGORY_ID == 'burglary')|(df_property.OFFENSE_CATEGORY_ID == 'auto-theft')].copy()
df_property_pm =  df_property.loc[df_property.TIME.str[-2:] == "PM" ].copy()
df_property_pm['TIME'] = [x[:-3] for x in df_property_pm['TIME']]
df_property_pm['TIME'] =  pd.to_datetime(df_property_pm['TIME'], format='%H:%M:%S')
df_property_pm['TIME'] = df_property_pm['TIME'] + timedelta(hours=12)


In [5]:
final_property = pd.concat([df_property_am, df_property_pm], ignore_index=True, sort =False)

block1 = '09/19/18 04:00:00'
block2 = '09/19/18 08:00:00'
block3 = '09/19/18 12:00:00'
block4 = '09/19/18 16:00:00'
block5 = '09/19/18 20:00:00'
block6 = '09/19/18 23:59:00'
datetime_object1 = datetime.strptime(block1, '%m/%d/%y %H:%M:%S')
datetime_object2 = datetime.strptime(block2, '%m/%d/%y %H:%M:%S')
datetime_object3 = datetime.strptime(block3, '%m/%d/%y %H:%M:%S')
datetime_object4 = datetime.strptime(block4, '%m/%d/%y %H:%M:%S')
datetime_object5 = datetime.strptime(block5, '%m/%d/%y %H:%M:%S')
datetime_object6 = datetime.strptime(block6, '%m/%d/%y %H:%M:%S')


In [6]:
final_property.loc[final_property['TIME'].dt.time <= datetime_object1.time(), 'TIME'] = datetime_object1

final_property.loc[(final_property['TIME'].dt.time > datetime_object1.time())&(final_property['TIME'].dt.time <= datetime_object2.time()), 'TIME'] = datetime_object2

final_property.loc[(final_property['TIME'].dt.time > datetime_object2.time())&(final_property['TIME'].dt.time <= datetime_object3.time()), 'TIME'] = datetime_object3

final_property.loc[(final_property['TIME'].dt.time > datetime_object3.time())&(final_property['TIME'].dt.time <= datetime_object4.time()), 'TIME'] = datetime_object4

final_property.loc[(final_property['TIME'].dt.time > datetime_object4.time())&(final_property['TIME'].dt.time <= datetime_object5.time()), 'TIME'] = datetime_object5

final_property.loc[(final_property['TIME'].dt.time > datetime_object5.time())&(final_property['TIME'].dt.time <= datetime_object6.time()), 'TIME'] = datetime_object6



final_property['TIME'] = [d.time() for d in final_property['TIME']]
final_property

,OFFENSE_CATEGORY_ID,TIME,day_of_week,NEIGHBORHOOD_ID,GEO_LON,GEO_LAT
0,theft-from-motor-vehicle,16:00:00,Wednesday,highland,-105.015451,39.757627
1,theft-from-motor-vehicle,08:00:00,Sunday,ruby-hill,-104.997720,39.694578
2,theft-from-motor-vehicle,04:00:00,Sunday,highland,-105.014275,39.764565
3,auto-theft,12:00:00,Saturday,west-colfax,-105.032978,39.737471
4,auto-theft,08:00:00,Monday,dia,-104.700816,39.831895
...,...,...,...,...,...,...
148113,theft-from-motor-vehicle,20:00:00,Tuesday,cole,-104.970490,39.770122
148114,larceny,20:00:00,Sunday,north-capitol-hill,-104.981335,39.746330
148115,auto-theft,20:00:00,Sunday,sunnyside,-105.011567,39.775002
148116,theft-from-motor-vehicle,20:00:00,Friday,speer,-104.974448,39.716818


In [8]:
property_csv = final_property[['day_of_week','NEIGHBORHOOD_ID', 'TIME']]
property_plot = final_property[['day_of_week','NEIGHBORHOOD_ID', 'TIME','GEO_LON','GEO_LAT']]
property_csv = property_csv.sample(20000)
property_csv.to_csv('Property.csv')
property_plot.to_csv('Property_Plot.csv')


!python apriori.py -f 'Property.csv' -s 0.002

item: ('23:59:00', 'congress-park') , 0.002
item: ('23:59:00', 'marston') , 0.002
item: ('23:59:00', 'hale') , 0.002
item: ('23:59:00', 'harvey-park-south') , 0.002
item: ('hampden-south', 'Sunday') , 0.002
item: ('auraria', '16:00:00') , 0.002
item: ('hampden-south', '04:00:00') , 0.002
item: ('university-hills', '16:00:00') , 0.002
item: ('northeast-park-hill', '12:00:00') , 0.002
item: ('Friday', 'mar-lee') , 0.002
item: ('23:59:00', 'bear-valley') , 0.002
item: ('Thursday', '16:00:00', 'stapleton') , 0.002
item: ('23:59:00', 'lincoln-park') , 0.002
item: ('elyria-swansea', 'Wednesday') , 0.002
item: ('east-colfax', '04:00:00') , 0.002
item: ('Tuesday', 'city-park-west') , 0.002
item: ('cheesman-park', '23:59:00') , 0.002
item: ('Tuesday', 'cherry-creek') , 0.002
item: ('east-colfax', 'Saturday') , 0.002
item: ('college-view-south-platte', '04:00:00') , 0.002
item: ('04:00:00', 'dia') , 0.002
item: ('north-capitol-hill', 'Tuesday') , 0.002
item: ('college-view-south-platte', 'Saturd